In [1]:
''' setting before run. every notebook should include this code. '''
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys

_r = os.getcwd().split('/')
_p = '/'.join(_r[:_r.index('gate-decorator-pruning')+1])
print('Change dir from %s to %s' % (os.getcwd(), _p))
os.chdir(_p)
sys.path.append(_p)

from config import parse_from_dict
parse_from_dict({
    "base": {
        "task_name": "resnet56_finetune",
        "cuda": True,
        "seed": 0,
        "checkpoint_path": "",
        "epoch": 0,
        "multi_gpus": True,
        "fp16": False
    },
    "model": {
        "name": "cifar.resnet56",
        "num_class": 10,
        "pretrained": False
    },
    "train": {
        "trainer": "normal",
        "max_epoch": 160,
        "optim": "sgd",
        "steplr": [
            [80, 0.1],
            [120, 0.01],
            [160, 0.001]
        ],
        "weight_decay": 5e-4,
        "momentum": 0.9,
        "nesterov": False
    },
    "data": {
        "type": "cifar10",
        "shuffle": True,
        "batch_size": 64,
        "test_batch_size": 128,
        "num_workers": 8
    },
    "loss": {
        "criterion": "softmax"
    },
    "gbn": {
        "finetune_epoch": 40,
        "lr_min": 1e-3,
        "lr_max": 1e-2
    }
})
from config import cfg

Change dir from /root/code/gate-decorator-pruning/run/resnet-56 to /root/code/gate-decorator-pruning
Parsing config file...
** Assert in demo mode. **


usage: ipykernel_launcher.py [-h] [--config CONFIG]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-ad976665-63c8-4e67-af41-4200de9de48d.json


In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim

from logger import logger
from main import set_seeds, recover_pack, adjust_learning_rate, _step_lr, _sgdr
from models import get_model
from utils import dotdict

from prune.universal import Meltable, GatedBatchNorm2d, Conv2dObserver, IterRecoverFramework, FinalLinearObserver
from prune.utils import analyse_model, finetune

In [3]:
set_seeds()
pack = recover_pack()

==> Preparing Cifar10 data..
Files already downloaded and verified
Files already downloaded and verified


/root/code/gate-decorator-pruning/models/cifar/resnet56.py:34: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


In [4]:
GBNs = GatedBatchNorm2d.transform(pack.net)
for gbn in GBNs:
    gbn.extract_from_bn()

-------

#### 70% flops reduced

In [5]:
model_dict = torch.load('./ckps/resnet56_cifar10_70percent_flops_reduced.ckp', map_location='cpu' if not cfg.base.cuda else 'cuda')
pack.net.module.load_state_dict(model_dict)

In [6]:
_ = Conv2dObserver.transform(pack.net.module)
pack.net.module.linear = FinalLinearObserver(pack.net.module.linear)
Meltable.observe(pack, 0.001)
Meltable.melt_all(pack.net)

In [7]:
pack.optimizer = optim.SGD(
    pack.net.parameters(),
    lr=1,
    momentum=cfg.train.momentum,
    weight_decay=cfg.train.weight_decay,
    nesterov=cfg.train.nesterov
)

In [8]:
pack.trainer.test(pack)

{'test_loss': 0.2779307614398908, 'acc@1': 91.79193037974683}

In [9]:
_ = finetune(pack, lr_min=cfg.gbn.lr_min, lr_max=cfg.gbn.lr_max, T=cfg.gbn.finetune_epoch)

  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.08736122048953, 'epoch_time': 41.72056436538696, 'test_loss': 0.2590870365877695, 'acc@1': 92.41495253164557, 'LR': 0.0014494245524296675}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.08371398926181409, 'epoch_time': 40.44017148017883, 'test_loss': 0.27484140394231943, 'acc@1': 92.23694620253164, 'LR': 0.0018994245524296676}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

...

{'train_loss': 0.03265679780574863, 'epoch_time': 39.70557951927185, 'test_loss': 0.2891138912756232, 'acc@1': 92.76107594936708, 'LR': 0.0019005754475703326}


  0%|                                                                       | 0/782 [00:00<?, ?it/s]

{'train_loss': 0.03141301143866823, 'epoch_time': 38.631301403045654, 'test_loss': 0.2897222333505184, 'acc@1': 92.7314082278481, 'LR': 0.001450575447570333}


100%|█████████████████████████████████████████████████████████████| 782/782 [00:38<00:00, 20.10it/s]


{'train_loss': 0.02813051395651782, 'epoch_time': 38.917781352996826, 'test_loss': 0.28455836571092846, 'acc@1': 92.85007911392405, 'LR': 0.0010005754475703327}


In [10]:
pack.net

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(1, 16, kernel_size=(